# Тестовое задание "МигКредит"

## Библиотеки

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore") 
from tqdm import tqdm_notebook
import os, re, sys, gc, pickle, time
from collections import defaultdict
from sklearn.metrics import make_scorer, roc_auc_score
import pandas as pd
import numpy as np
from itertools import combinations
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
from plot_metric.functions import BinaryClassification
import warnings
warnings.filterwarnings("ignore") 
from tqdm import tqdm_notebook
import os, re, sys, gc, pickle, time
from collections import defaultdict
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
from sklearn.model_selection import GridSearchCV, cross_val_score,\
                                    KFold, train_test_split, cross_validate, ParameterGrid,\
                                    cross_validate, cross_val_predict, TimeSeriesSplit, StratifiedKFold
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import statsmodels.api as sm
from sklearn.base import BaseEstimator, TransformerMixin,  clone
from sklearn.metrics import confusion_matrix
# пайплайн
from sklearn.pipeline import Pipeline, make_union, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin,  clone
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
# дамми-регрессор
from sklearn.dummy import DummyClassifier
from itertools import combinations
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import time
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_curve
import shutil
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Вспомогательные функции

In [2]:
def convert_types(df):
    '''конвертирует типы (по возможности)'''
    df_c = df.copy()
    for col in df.columns:
        ser = df[col]
        try:
            ser2 = ser.astype('datetime64')
            df_c[col] = ser2
        except:
            try:
                ser_float =ser.astype(np.float32)
                ser_int =ser.astype(np.int32)
                if (ser_float == ser_int).all():
                    df_c[col] =ser_int
                else:
                    df_c[col] =ser_float
            except:
                try:
                    df_c[col] = ser.str.lower().str.strip()
                except:
                    df_c[col] = ser
    return df_c

def clean_CREDITTOTALAMT(df):
    df_c = df.copy()

    df_c['CREDITTOTALAMT'] = \
        df_c['CREDITTOTALAMT'].apply(lambda x: str(x).replace(',', '').replace('.', '').strip())
    uniques = df_c['CREDITTOTALAMT'].unique()
    d = {}
    for val in uniques:
        try:
            d[val] = np.int32(re.findall('[0-9]+', val)[0])
        except:
            pass
    df_c['CREDITTOTALAMT'] = df_c['CREDITTOTALAMT'].map(d)
    return df_c

def collect_features_v2(df_buffer_table2_tr, idx_column ='REQUEST_ID'):
    
    ##################################################################################################
    obj_columns = ['ownerIndicText', 'acctTypeText', 'currencyCode', 'paymtFreqText',\
                   'accountRatingText', 'interestPaymentFrequencyText','BUSINESSCATEGORY']
    cat_columns = ['interestPaymentFrequencyCode', 'termsFrequency', 'acctType',  'accountRating',
                   'numDays30', 'numDays60', 'numDays90', 'monthsReviewed']
    cat_columns2 = ['numDays30', 'numDays60', 'numDays90', 'numDays120', 'numDays30.//7',
                    'numDays30.%7', 'numDays60.//7', 'numDays60.%7', 'numDays90.//7',
                    'numDays90.%7', 'monthsReviewed', 'monthsReviewed.//7', 'monthsReviewed.%7']
    dt_columns = ['openedDt', 'lastPaymtDt', 'closedDt', 'reportingDt', 'paymtPatStartDt',
                  'lastUpdatedDt', 'accountRatingDate', 'paymentDueDate',
                  'interestPaymentDueDate']
    num_columns  = ['creditLimit', 'curBalanceAmt', 'amtPastDue', 'termsAmt',
                    'amtOutstanding', 'CREDITTOTALAMT', 'PRINCIPALOUTSTANDING',
                    'PRINCIPALPASTDUE', 'OTHERAMTOUTSTANDING', 'OTHERAMTPASTDUE',
                    'INTOUTSTANDING', 'INTPASTDUE']
    ##################################################################################################
    
    df_features = pd.DataFrame()
    for REQUEST_ID, SUBDF in tqdm_notebook(df_buffer_table2_tr.groupby(idx_column)):

        nrows = SUBDF.shape[0]   
        df_features.loc[REQUEST_ID, 'nrecords'] = nrows     

        for col in SUBDF.columns:
            ser = SUBDF[col]
            try:
                nuniques = ser.nunique()
                df_features.loc[REQUEST_ID, f'{col}.nuniques1'] = nuniques
                df_features.loc[REQUEST_ID, f'{col}.nuniques2'] = nuniques/nrows
            except:
                df_features.loc[REQUEST_ID, f'{col}.nuniques1'] = np.nan
                df_features.loc[REQUEST_ID, f'{col}.nuniques2'] = nuniques/nrows

            try:
                df_features.loc[REQUEST_ID, f'{col}.nanshare'] = ser.isna().mean()
            except:
                df_features.loc[REQUEST_ID, f'{col}.nanshare'] = np.nan

        for col in dt_columns:
            ser = SUBDF[col]
            duration = (ser.max()-ser.min()) / np.timedelta64(1, 'D')
            df_features.loc[REQUEST_ID, f'{col}.duration'] = duration

            _y_shares = ser.dt.year.value_counts(normalize = True)
            _m_shares = ser.dt.month.value_counts(normalize = True)
            _d_shares = ser.dt.day.value_counts(normalize = True)
            _dow_shares = ser.dt.dayofweek.value_counts(normalize = True)

            for name, share_ser in zip(('y', 'm', 'd', 'dow'), (_y_shares, _m_shares, _d_shares, _dow_shares)):
                for k, v in share_ser.items():
                    df_features.loc[REQUEST_ID, f'{col}.{name}.{k}.share'] = v 

            uniques_sorted = ser.sort_values().unique()
            L_diffs = []
            for i in range(len(uniques_sorted)):
                for j in range(i+1, len(uniques_sorted)):
                    ts1, ts2 = uniques_sorted[i], uniques_sorted[j]
                    diff = (ts1-ts2)//np.timedelta64(1,'D')
                    L_diffs.append(diff)
            diffs_ser = pd.Series(L_diffs).abs()
            step=7
            diff_bins = pd.Series({i:diffs_ser.between(i*(step), (i+1)*step).mean() for i in range(34)})
            diff_bins[-1] = (diffs_ser==0).mean()
            for k, v in diff_bins.items():
                df_features.loc[REQUEST_ID, f'{col}.diff{k}.share'] = v

        for l in (obj_columns, cat_columns, cat_columns2):
            for col in l:
                shares = SUBDF[col].value_counts(normalize = True)
                for k, v in shares.items():
                    df_features.loc[REQUEST_ID, f'{col}.{k}.share'] = v             

        for col in ['UUID', 'paymtPat']:
            ser = SUBDF[col]
            df_features.loc[REQUEST_ID, col] = '&'.join(list(map(str, ser)))
            df_features.loc[REQUEST_ID, f'{col}.uniques'] = '&'.join(list(map(str, ser.unique())))  
            
        SUBDF_NUM_LOG = SUBDF[num_columns].apply(np.log1p)
        try:
            for col, value in SUBDF_NUM_LOG.mean().items():
                df_features.loc[REQUEST_ID, f'{col}.mean'] = value
        except:
            pass
        try:
            for col, value in SUBDF_NUM_LOG.max().items():
                df_features.loc[REQUEST_ID, f'{col}.max'] = value
        except:
            pass
        try:
            for col, value in SUBDF_NUM_LOG.min().items():
                df_features.loc[REQUEST_ID, f'{col}.min'] = value
        except:
            pass   

        SUBDF_NUM_LOG_ROUND = SUBDF_NUM_LOG.round()
        for col in SUBDF_NUM_LOG_ROUND.columns:
            ser = SUBDF_NUM_LOG_ROUND[col]
            shares = ser.value_counts(normalize = True)
            for k, v in shares.items():
                df_features.loc[REQUEST_ID, f'{col}.{k}.share'] = v            
        
    return df_features.reset_index().rename(columns = {'index':idx_column})

def collect_features_v3(df_buffer_table3_tr, idx_column ='REQUEST_ID'):
    cat_columns = ['inquiryPeriod', 'inqPurposeText', 'currencyCode']
    num_columns = ['inqAmount']
    df_features = pd.DataFrame()
    for REQUEST_ID, SUBDF in tqdm_notebook(df_buffer_table3_tr.groupby(idx_column)):
        
        nrows = SUBDF.shape[0]
        df_features.loc[REQUEST_ID, 'nrecords_v2'] = nrows 
        
        for col in SUBDF.columns:
            ser = SUBDF[col]
            try:
                nuniques = ser.nunique()
                df_features.loc[REQUEST_ID, f'{col}.nuniques1'] = nuniques
                df_features.loc[REQUEST_ID, f'{col}.nuniques2'] = nuniques/nrows
            except:
                df_features.loc[REQUEST_ID, f'{col}.nuniques1'] = np.nan
                df_features.loc[REQUEST_ID, f'{col}.nuniques2'] = nuniques/nrows

            try:
                df_features.loc[REQUEST_ID, f'{col}.nanshare'] = ser.isna().mean()
            except:
                df_features.loc[REQUEST_ID, f'{col}.nanshare'] = np.nan
                
        for col in cat_columns:
            ser = SUBDF[col]
            shares = ser.value_counts(normalize = True)
            for k, v in shares.items():
                df_features.loc[REQUEST_ID, f'{col}.{k}.share'] = v
        
        SUBDF_NUM_LOG = SUBDF[num_columns].apply(np.log1p)
        try:
            for col, value in SUBDF_NUM_LOG.mean().items():
                df_features.loc[REQUEST_ID, f'{col}.mean'] = value
        except:
            pass
        try:
            for col, value in SUBDF_NUM_LOG.max().items():
                df_features.loc[REQUEST_ID, f'{col}.max'] = value
        except:
            pass
        try:
            for col, value in SUBDF_NUM_LOG.min().items():
                df_features.loc[REQUEST_ID, f'{col}.min'] = value
        except:
            pass   

        SUBDF_NUM_LOG_ROUND = SUBDF_NUM_LOG.round()
        for col in SUBDF_NUM_LOG_ROUND.columns:
            ser = SUBDF_NUM_LOG_ROUND[col]
            shares = ser.value_counts(normalize = True)
            for k, v in shares.items():
                df_features.loc[REQUEST_ID, f'{col}.{k}.share'] = v    
    return df_features.reset_index().rename(columns = {'index':idx_column})

class SklearnHelperFeatureSelector(BaseEstimator, TransformerMixin):
    ''' Отбор признаков '''
    def __init__(self, model, cv, scoring, show_progress):
        self.model = model
        self.cv = cv
        self.scoring = scoring
        self.show_progress = show_progress
    def fit(self, X, y=None):
        #assert (isinstance(X, np.ndarray)) or (X.getformat() == 'csc')
        _X = X.copy()
        cv_scores = []
        for i in tqdm_notebook(range(_X.shape[1])):
            try:
                _X_curr = _X[:, i].toarray().reshape(-1,1)
            except:
                _X_curr = _X[:, i].reshape(-1,1)                
            mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()            
            cv_scores.append(mean_cv_score)
            
        order = np.argsort(cv_scores)[::-1]
        to_drop_before, best_features, best_cv_score = [], [order[0]], -np.inf
        for i in tqdm_notebook(order[1:]):
            curr_features = best_features+[i]
            _X_curr = _X[:, curr_features]
            mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()
            if mean_cv_score>best_cv_score:
                best_cv_score = mean_cv_score
                best_features = curr_features
                if self.show_progress:
                    print('new best score = {:.10f}'.format(best_cv_score))
            else:
                to_drop_before.append(i)
        while True:
            to_drop_after = []
            for i in tqdm_notebook(to_drop_before):
                curr_features = best_features+[i]
                _X_curr = _X[:, curr_features]
                mean_cv_score = cross_val_score(self.model, _X_curr, y, cv =self.cv, scoring = self.scoring, n_jobs=-1).mean()
                if mean_cv_score>best_cv_score:
                    best_cv_score = mean_cv_score
                    best_features = curr_features
                    if self.show_progress:
                        print('new best score = {:.10f}'.format(best_cv_score))
                else:
                    to_drop_after.append(i)
            if to_drop_before == to_drop_after:
                break
            else:
                to_drop_before = to_drop_after  
        self.best_features_ = best_features
        self.best_score_ = best_cv_score
    def transform(self, _X):
        return _X[:, self.best_features_]
    
class SklearnHelperReduceMulticollinearity(BaseEstimator, TransformerMixin):
    ''' сокращение мультиколлинеарности '''
    def __init__(self, treshold):
        self.treshold = treshold        
    def fit(self, X, y=None):
        self.D = defaultdict(list)
        for i in tqdm_notebook(range(X.shape[1])):
            for j in range(i+1, X.shape[1]):
                x1, x2 = X[:, i], X[:, j]
                idxs = [i, j]
                
                corr_x1x2 = np.abs(np.corrcoef(x1, x2)).min()
                
                corr_x1y = np.abs(np.corrcoef(x1, y)).min()
                corr_x2y = np.abs(np.corrcoef(x2, y)).min()
                to_keep =idxs[np.argmax(np.array([corr_x1y, corr_x2y]))]
                
                if corr_x1x2 >= self.treshold:
                    self.D[i].append(to_keep)
        return self
    def transform(self, X):
        pass 
def calc_vif(X): 
    _X = pd.DataFrame(X)
    vif = pd.DataFrame()
    vif["variables"] = _X.columns
    vif["VIF"] = [variance_inflation_factor(_X.values, i) for i in tqdm_notebook(range(_X.shape[1]))]
    return(vif)

## Подготовка данных

In [3]:
df_buffer_table1 = pd.read_csv('buffer_table1.csv', sep=',').iloc[:, 1:]\
                                   .rename(columns ={'30+ 3MoB gt 0':'target'})
df_buffer_table2= pd.read_csv('buffer_table2.csv', sep=',', encoding = 'cp1251').iloc[:, 1:]\
                                  .dropna(subset = ['REQUEST_ID'])
df_buffer_table3= pd.read_csv('buffer_table3.csv', encoding = 'cp1251').iloc[:, 1:]\
                                  .dropna(subset = ['REQUEST_ID'])
df_buffer_table2 = clean_CREDITTOTALAMT(df_buffer_table2)
df_buffer_table1 = convert_types(df_buffer_table1)
df_buffer_table2 = convert_types(df_buffer_table2)
df_buffer_table3 = convert_types(df_buffer_table3)

In [7]:
df_buffer_table1_tr = df_buffer_table1[df_buffer_table1['train']==1]
df_buffer_table1_te = df_buffer_table1[df_buffer_table1['train']!=1]

idx_train = df_buffer_table1_tr['cash_hist'].unique()
idx_test = df_buffer_table1_te['cash_hist'].unique()
df_buffer_table2_tr = df_buffer_table2[df_buffer_table2['REQUEST_ID'].isin(idx_train)]
df_buffer_table2_te = df_buffer_table2[df_buffer_table2['REQUEST_ID'].isin(idx_test)]

In [11]:
obj_columns = ['ownerIndicText', 'acctTypeText', 'currencyCode', 'paymtPat',
               'paymtFreqText', 'accountRatingText', 'interestPaymentFrequencyText',
               'UUID', 'BUSINESSCATEGORY']
for col in tqdm_notebook(obj_columns):
    unique_values = df_buffer_table2_tr[col].unique()
    unique_labels = range(len(unique_values))
    d = dict(zip(unique_values, unique_labels))
    df_buffer_table2_tr[col] =df_buffer_table2_tr[col].map(d)
    df_buffer_table2_te[col] =df_buffer_table2_te[col].map(d)
    
cat_columns2 = ['numDays30','numDays60', 'numDays90', 'monthsReviewed']
for col in cat_columns2:
    df_buffer_table2_tr[f'{col}.//7'] =df_buffer_table2_tr[col]//7
    df_buffer_table2_tr[f'{col}.%7'] =df_buffer_table2_tr[col]%7
    
    df_buffer_table2_te[f'{col}.//7'] =df_buffer_table2_te[col]//7
    df_buffer_table2_te[f'{col}.%7'] =df_buffer_table2_te[col]%7    

## Извлечение признаков

In [15]:
features2_tr = collect_features_v2(df_buffer_table2_tr, idx_column ='REQUEST_ID')
features2_tr.to_pickle('features2_tr.pickle')
features2_te = collect_features_v2(df_buffer_table2_te, idx_column ='REQUEST_ID')
features2_te.to_pickle('features2_te.pickle')

vec12 = TfidfVectorizer(max_features = 50)

vec12.fit(features2_tr['UUID'])
UUID_tfidf_tr = vec12.transform(features2_tr['UUID'])
UUID_tfidf_te = vec12.transform(features2_te['UUID'])

vec12.fit(features2_tr['UUID.uniques'])
UUID_unqiues_tfidf_tr = vec12.transform(features2_tr['UUID.uniques'])
UUID_unqiues_tfidf_te = vec12.transform(features2_te['UUID.uniques'])

vec12.fit(features2_tr['paymtPat'])
paymtPat_tfidf_tr = vec12.transform(features2_tr['paymtPat'])
paymtPat_tfidf_te = vec12.transform(features2_te['paymtPat'])

vec12.fit(features2_tr['paymtPat.uniques'])
paymtPat_uniques_tfidf_tr = vec12.transform(features2_tr['paymtPat.uniques'])
paymtPat_uniques_tfidf_te = vec12.transform(features2_te['paymtPat.uniques'])

features2_tr = features2_tr.drop(['UUID', 'UUID.uniques', 'paymtPat', 'paymtPat.uniques'], 1)
features2_te = features2_te.drop(['UUID', 'UUID.uniques', 'paymtPat', 'paymtPat.uniques'], 1)
_df1_tr = pd.DataFrame(UUID_tfidf_tr.toarray(),\
             index = features2_tr.index,\
             columns = [f'UUID{i}' for i in range(UUID_tfidf_tr.shape[1])])
_df2_tr = pd.DataFrame(UUID_unqiues_tfidf_tr.toarray(),\
             index = features2_tr.index,\
             columns = [f'UUID_unqiues{i}' for i in range(UUID_unqiues_tfidf_tr.shape[1])])
_df3_tr = pd.DataFrame(paymtPat_tfidf_tr.toarray(),\
             index = features2_tr.index,\
             columns = [f'paymtPat{i}' for i in range(paymtPat_tfidf_tr.shape[1])])
_df4_tr = pd.DataFrame(paymtPat_uniques_tfidf_tr.toarray(),\
             index = features2_tr.index,\
             columns = [f'paymtPat_uniques{i}' for i in range(paymtPat_uniques_tfidf_tr.shape[1])])
_df_encs_tr = pd.concat([_df1_tr, _df2_tr, _df3_tr, _df4_tr], 1)

_df1_te = pd.DataFrame(UUID_tfidf_te.toarray(),\
             index = features2_te.index,\
             columns = [f'UUID{i}' for i in range(UUID_tfidf_te.shape[1])])
_df2_te = pd.DataFrame(UUID_unqiues_tfidf_te.toarray(),\
             index = features2_te.index,
             columns = [f'UUID_unqiues{i}' for i in range(UUID_unqiues_tfidf_te.shape[1])])
_df3_te = pd.DataFrame(paymtPat_tfidf_te.toarray(),\
             index = features2_te.index,
             columns = [f'paymtPat{i}' for i in range(paymtPat_tfidf_te.shape[1])])
_df4_te = pd.DataFrame(paymtPat_uniques_tfidf_te.toarray(),\
             index = features2_te.index,
             columns = [f'paymtPat_uniques{i}' for i in range(paymtPat_uniques_tfidf_te.shape[1])])
_df_encs_te = pd.concat([_df1_te, _df2_te, _df3_te, _df4_te], 1)

features2_full_tr = pd.concat([features2_tr, _df_encs_tr], 1)
features2_full_te = pd.concat([features2_te, _df_encs_te], 1)
features2_full_tr.to_pickle('features2_full_tr.pickle')
features2_full_te.to_pickle('features2_full_te.pickle')

In [ ]:
df_buffer_table3_tr = df_buffer_table3[df_buffer_table3['REQUEST_ID'].isin(idx_train)]
df_buffer_table3_te = df_buffer_table3[df_buffer_table3['REQUEST_ID'].isin(idx_test)]

features3_tr = collect_features_v3(df_buffer_table3_tr, idx_column ='REQUEST_ID')
features3_tr.to_pickle('features3_tr.pickle')
features3_te = collect_features_v3(df_buffer_table3_te, idx_column ='REQUEST_ID')
features3_te.to_pickle('features3_te.pickle')

## Подготовка финальных таблиц

In [60]:
X2_tr = pd.merge(df_buffer_table1_tr[['ucdb_id', 'cash_hist']],\
                 features2_full_tr,\
                 left_on = 'cash_hist', right_on='REQUEST_ID',\
                 how='left').set_index('ucdb_id')\
          .drop(['cash_hist','REQUEST_ID'], 1)
X2_te = pd.merge(df_buffer_table1_te[['ucdb_id', 'cash_hist']],\
                 features2_full_te,\
                 left_on = 'cash_hist', right_on='REQUEST_ID',\
                 how='left').set_index('ucdb_id')\
          .drop(['cash_hist','REQUEST_ID'], 1)
X3_tr = pd.merge(df_buffer_table1_tr[['ucdb_id', 'cash_hist']],\
                 features3_tr,\
                 left_on = 'cash_hist', right_on='REQUEST_ID',\
                 how='left').set_index('ucdb_id')\
          .drop(['cash_hist','REQUEST_ID'], 1)
X3_te = pd.merge(df_buffer_table1_te[['ucdb_id', 'cash_hist']],\
                 features3_te,\
                 left_on = 'cash_hist', right_on='REQUEST_ID',\
                 how='left').set_index('ucdb_id')\
          .drop(['cash_hist','REQUEST_ID'], 1)
X_tr = pd.concat([X2_tr, X3_tr], 1)
X_te = pd.concat([X2_te, X3_te], 1)
feat_to_use = list(set(X_te.columns.tolist()).intersection(set(X_tr.columns.tolist())))
X_tr = X_tr[feat_to_use]
X_te = X_te[feat_to_use]
X_tr.to_pickle('X_tr.pickle')
X_te.to_pickle('X_te.pickle')
y_tr = df_buffer_table1_tr.set_index('ucdb_id')['target']
y_te = df_buffer_table1_te.set_index('ucdb_id')['target']
y_tr.to_pickle('y_tr.pickle')
y_te.to_pickle('y_te.pickle')

## Моделирование

In [42]:
X_tr = pd.read_pickle('X_tr.pickle')
X_te = pd.read_pickle('X_te.pickle')
y_tr = pd.read_pickle('y_tr.pickle')
y_te = pd.read_pickle('y_te.pickle')

In [5]:
SEED = 13
logit = LogisticRegression(random_state= SEED)
SKF = StratifiedKFold(5, shuffle = True, random_state = SEED)
N_ITERATIONS=100
ALPHA= .05

ppl = make_pipeline(MinMaxScaler(),\
                    SimpleImputer(strategy='constant', fill_value=-1),\
                    SklearnHelperFeatureSelector(model=logit, cv=SKF, scoring='roc_auc', show_progress=True))

In [6]:
ppl.fit(X_tr, y_tr)

new best score = 0.6540014783
new best score = 0.6782691121
new best score = 0.6801423928
new best score = 0.6979574673
new best score = 0.7185899727
new best score = 0.7325748571
new best score = 0.7342948988
new best score = 0.7365008592
new best score = 0.7398336150
new best score = 0.7402531280
new best score = 0.7426385793
new best score = 0.7427936207
new best score = 0.7442398949
new best score = 0.7504752795
new best score = 0.7506560888
new best score = 0.7509822932
new best score = 0.7527819584
new best score = 0.7536973702
new best score = 0.7537580850
new best score = 0.7538351217
new best score = 0.7542278321
new best score = 0.7544482612
new best score = 0.7551879372
new best score = 0.7551966155
new best score = 0.7562047685
new best score = 0.7569582630
new best score = 0.7580818208
new best score = 0.7586956446
new best score = 0.7595062824
new best score = 0.7605792724
new best score = 0.7607131020
new best score = 0.7618431518
new best score = 0.7619040002
new best s

new best score = 0.8327395281
new best score = 0.8331981770
new best score = 0.8336773033
new best score = 0.8337472304
new best score = 0.8340231507
new best score = 0.8342032090
new best score = 0.8345484223
new best score = 0.8345922979
new best score = 0.8348632281



Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('simpleimputer',
                 SimpleImputer(fill_value=-1, strategy='constant')),
                ('sklearnhelperfeatureselector',
                 SklearnHelperFeatureSelector(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
                                              model=LogisticRegression(random_state=13),
                                              scoring='roc_auc',
                                              show_progress=True))])

In [7]:
_x_selected_tr = ppl.transform(X_tr)
_x_selected_te = ppl.transform(X_te)
_y_tr = y_tr.values
_y_te = y_te.values

In [8]:
# with open('_x_selected_tr.pickle', 'wb') as f:
#     pickle.dump(_x_selected_tr, f)
# with open('_x_selected_te.pickle', 'wb') as f:
#     pickle.dump(_x_selected_te, f)
# with open('_y_tr.pickle', 'wb') as f:
#     pickle.dump(_y_tr, f)
# with open('_y_te.pickle', 'wb') as f:
#     pickle.dump(_y_te, f)

In [34]:
with open('_x_selected_tr.pickle', 'rb') as f:
    _x_selected_tr =pickle.load(f)
with open('_x_selected_te.pickle', 'rb') as f:
    _x_selected_te =pickle.load(f)
with open('_y_tr.pickle', 'rb') as f:
    _y_tr =pickle.load(f)
with open('_y_te.pickle', 'rb') as f:
    _y_te =pickle.load(f)

In [37]:
VIF=100
sigLevel = 0.05
SEED = 13
logit = LogisticRegression(random_state= SEED)
SKF = StratifiedKFold(5, shuffle = True, random_state = SEED)
N_ITERATIONS=100
ALPHA= .05
treshold=.95

In [36]:
vif = calc_vif(_x_selected_tr)

In [38]:
iteration = 0
while True:
    print('iteration {}'.format(iteration))    
    idxs_to_use = vif[vif['VIF'] < VIF]['variables'].values
    _x2_selected_tr = _x_selected_tr[:, idxs_to_use]
    _x2_selected_te = _x_selected_te[:, idxs_to_use]

    corr_abs = pd.DataFrame(_x2_selected_tr).corr().abs()
    corr_abs_triu = corr_abs.where(np.triu(np.ones_like(corr_abs).astype(np.bool), k=1))
    if  (corr_abs_triu.stack()>=treshold).any():
        VIF-=1
        iteration+=1
    else:
        break
    
X_opt_tr = _x2_selected_tr
X_opt_te = _x2_selected_te
sm_logit = sm.Logit(_y_tr,X_opt_tr).fit()
pVals = sm_logit.pvalues

while pVals[np.argmax(pVals)] > sigLevel:    
    X_opt_tr = np.delete(X_opt_tr, np.argmax(pVals), axis = 1)    
    X_opt_te = np.delete(X_opt_te, np.argmax(pVals), axis = 1) 
    sm_logit = sm.Logit(_y_tr,X_opt_tr).fit()
    pVals = sm_logit.pvalues
sm_logit = sm.Logit(_y_tr,X_opt_tr).fit()

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

Optimization terminated successfully.
         Current function value: 0.255436
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.255629
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.255844
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.256057
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.256273
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.256513
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.256691
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.256922
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.257158
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.257415
  

In [39]:
sm_logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 5632
Model:                          Logit   Df Residuals:                     5583
Method:                           MLE   Df Model:                           48
Date:                Sat, 03 Jul 2021   Pseudo R-squ.:                  0.1686
Time:                        16:48:44   Log-Likelihood:                -1483.1
converged:                       True   LL-Null:                       -1783.8
Covariance Type:            nonrobust   LLR p-value:                 1.052e-96
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7321      0.116      6.302      0.000       0.504       0.960
x2             1.4063      0.365      3.853      0.000       0.691       2.122
x3             1.4942      0.392      3.812      0.000       0.726       2.262
x4            -1.2033      0.392     -3.073      0.002      -1.971      -0.436
x5             0.2399      0.116      2.071      0.038       0.013       0.467
x6             0.3688      0.104      3.552      0.000       0.165       0.572
x7             0.3191      0.123      2.589      0.010       0.078       0.561
x8             0.4725      0.196      2.412      0.016       0.089       0.856
x9            -1.2203      0.487     -2.504      0.012      -2.175      -0.265
x10           -0.5202      0.132     -3.947      0.000      -0.779      -0.262
x11            0.7520      0.288      2.613      0.009       0.188       1.316
x12            0.2706      0.105      2.571      0.010       0.064       0.477
x13            0.4438      0.127      3.492      0.000       0.195       0.693
x14            0.3909      0.135      2.889      0.004       0.126       0.656
x15            0.3408      0.108      3.160      0.002       0.129       0.552
x16            0.3486      0.104      3.352      0.001       0.145       0.552
x17            0.2628      0.103      2.541      0.011       0.060       0.465
x18            0.3171      0.138      2.306      0.021       0.048       0.587
x19            0.5262      0.121      4.332      0.000       0.288       0.764
x20           -0.2781      0.112     -2.488      0.013      -0.497      -0.059
x21           -6.3852      1.425     -4.481      0.000      -9.178      -3.593
x22            0.2865      0.103      2.783      0.005       0.085       0.488
x23           -0.2283      0.110     -2.079      0.038      -0.444      -0.013
x24           -1.1224      0.489     -2.295      0.022      -2.081      -0.164
x25           -0.8959      0.346     -2.592      0.010      -1.573      -0.218
x26            0.6958      0.232      3.003      0.003       0.242       1.150
x27           -0.5757      0.174     -3.312      0.001      -0.916      -0.235
x28            0.2831      0.126      2.242      0.025       0.036       0.531
x29           -0.5332      0.115     -4.641      0.000      -0.758      -0.308
x30           -0.2392      0.109     -2.191      0.028      -0.453      -0.025
x31           -0.2523      0.109     -2.324      0.020      -0.465      -0.040
x32           -0.5925      0.144     -4.128      0.000      -0.874      -0.311
x33           -0.3267      0.108     -3.016      0.003      -0.539      -0.114
x34            0.2051      0.100      2.045      0.041       0.008       0.402
x35           -0.2125      0.107     -1.981      0.048      -0.423      -0.002
x36           -0.3921      0.164     -2.389      0.017      -0.714      -0.070
x37           -4.7910      1.162     -4.125      0.000      -7.068      -2.514
x38           -1.2621      0.454     -2.782      0.005      -2.151      -0.373
x3

In [40]:
logit_gs = GridSearchCV(LogisticRegression(),\
                         param_grid = {'C':np.logspace(-3, 3, 500),
                                        'random_state': [SEED],\
                                        'solver' : ['liblinear'],\
                                        'penalty':['l1','l2']},\
                        scoring ='roc_auc', cv = SKF, verbose = 1,n_jobs=-1)
logit_gs.fit(X_opt_tr, _y_tr)
GINI_best_cv = 100*((2 *logit_gs.best_score_)-1)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    6.7s


[Parallel(n_jobs=-1)]: Done 2346 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done 2696 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 3146 tasks      | elapsed:   41.2s
[Parallel(n_jobs=-1)]: Done 3696 tasks      | elapsed:   55.9s
[Parallel(n_jobs=-1)]: Done 4346 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  1.5min finished


In [41]:
GINI_best_cv

54.42516356951028

In [47]:
y_predprob_te = pd.DataFrame(logit_gs.best_estimator_.predict_proba(X_opt_te)[:,1], columns= ['predprob'])
y_predprob_te['ucdb_id'] = y_te.index
y_predprob_te = y_predprob_te[['ucdb_id', 'predprob']]

In [54]:
y_predprob_te.to_excel('y_predprob_te_v4.xlsx')